In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from datetime import datetime

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
df = pd.read_csv('data/Chicago/Crimes_1year.csv',
                 header=0)
print('df.shape:', df.shape)
df.head()

df.shape: (264154, 30)


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards,Boundaries - ZIP Codes,Police Districts,Police Beats
0,11658366,JC227538,04/15/2019 11:59:00 PM,074XX N GREENVIEW AVE,0820,THEFT,$500 AND UNDER,VEHICLE NON-COMMERCIAL,False,False,2422,24,49.0,1.0,06,1164996.0,1949669.0,2019,04/22/2019 04:16:27 PM,42.017499,-87.668188,"(42.017499124, -87.668187508)",3.0,21853.0,10.0,357.0,5.0,9.0,11.0,32.0
1,11656397,JC226102,04/15/2019 11:57:00 PM,040XX W ARTHINGTON ST,1090,ARSON,ATTEMPT ARSON,STREET,False,False,1132,11,24.0,26.0,09,1149639.0,1895808.0,2019,04/22/2019 04:16:27 PM,41.870014,-87.726102,"(41.870013763, -87.726102154)",36.0,21572.0,27.0,703.0,14.0,30.0,16.0,142.0
2,11656390,JC226118,04/15/2019 11:55:00 PM,019XX W MONTEREY AVE,0460,BATTERY,SIMPLE,POLICE FACILITY/VEH PARKING LOT,True,False,2212,22,19.0,75.0,08B,1165742.0,1830843.0,2019,04/22/2019 04:16:27 PM,41.691414,-87.668826,"(41.691413679, -87.668826246)",33.0,22212.0,74.0,378.0,42.0,13.0,9.0,257.0
3,11656380,JC226099,04/15/2019 11:51:00 PM,076XX S STEWART AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,621,6,17.0,69.0,08B,1174949.0,1854311.0,2019,04/22/2019 04:16:27 PM,41.755613,-87.634421,"(41.755613227, -87.634420924)",17.0,21554.0,67.0,495.0,31.0,59.0,20.0,228.0
4,11656356,JC226096,04/15/2019 11:47:00 PM,077XX S NORMAL AVE,1822,NARCOTICS,MANU/DEL:CANNABIS OVER 10 GMS,STREET,True,False,621,6,17.0,69.0,18,1174301.0,1853682.0,2019,04/22/2019 04:16:27 PM,41.753902,-87.636814,"(41.753901595, -87.636814339)",17.0,21554.0,67.0,495.0,31.0,59.0,20.0,229.0


### To get more granularity with the crime type, we can do NLP on Description and tie it to severity of the crime
### Let's leave this for after the MVP

In [13]:
df.Description.unique()

array(['$500 AND UNDER', 'ATTEMPT ARSON', 'SIMPLE',
       'DOMESTIC BATTERY SIMPLE', 'MANU/DEL:CANNABIS OVER 10 GMS',
       'FALSE/STOLEN/ALTERED TRP', 'STRONGARM - NO WEAPON',
       'POSS: COCAINE', 'ARMED: OTHER DANGEROUS WEAPON',
       'ILLEGAL USE CASH CARD', 'AGGRAVATED: HANDGUN', 'TO VEHICLE',
       'THEFT BY LESSEE,MOTOR VEH', 'HARASSMENT BY ELECTRONIC MEANS',
       'AGGRAVATED DOMESTIC BATTERY: HANDS/FIST/FEET SERIOUS INJURY',
       'FORCIBLE ENTRY', 'CRIMINAL DEFACEMENT', 'POSS: PCP', 'TO LAND',
       'AGGRAVATED: OTHER DANG WEAPON', 'FIRST DEGREE MURDER',
       'NON-AGGRAVATED', 'OVER $500', 'AUTOMOBILE',
       'HARASSMENT BY TELEPHONE', 'UNAUTHORIZED VIDEOTAPING',
       'PURSE-SNATCHING', 'FRAUD OR CONFIDENCE GAME', 'TO PROPERTY',
       'UNLAWFUL ENTRY', 'UNLAWFUL POSS OF HANDGUN', 'CREDIT CARD FRAUD',
       'RETAIL THEFT', 'OTHER', 'VIOLATE ORDER OF PROTECTION',
       'AGGRAVATED VEHICULAR HIJACKING', 'ATTEMPT: ARMED-HANDGUN',
       'TELEPHONE THREAT', 'RECKL

In [14]:
df['Location Description'].nunique()

127

In [15]:
df.IUCR.nunique()

323

In [16]:
df.isnull().sum()

ID                               0
Case Number                      0
Date                             0
Block                            0
IUCR                             0
Primary Type                     0
Description                      0
Location Description           819
Arrest                           0
Domestic                         0
Beat                             0
District                         0
Ward                             5
Community Area                   2
FBI Code                         0
X Coordinate                  3092
Y Coordinate                  3092
Year                             0
Updated On                       0
Latitude                      3092
Longitude                     3092
Location                      3092
Historical Wards 2003-2015    3981
Zip Codes                     3092
Community Areas               3862
Census Tracts                 3701
Wards                         3860
Boundaries - ZIP Codes        3861
Police Districts    

In [17]:
print('Shape before dropping:', df.shape)
df = df.dropna(axis=0, how='any')
print('Shape after dropping:', df.shape)

Shape before dropping: (264154, 30)
Shape after dropping: (259204, 30)


### Remove columns we're not interested in

In [18]:
df = df.drop(columns=['ID', 'Case Number', 'IUCR',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards', 'Boundaries - ZIP Codes',
       'Police Districts', 'Police Beats'])
print('df.shape:', df.shape)
df.head()

df.shape: (259204, 5)


,Date,Block,Primary Type,Latitude,Longitude
0,04/15/2019 11:59:00 PM,074XX N GREENVIEW AVE,THEFT,42.017499,-87.668188
1,04/15/2019 11:57:00 PM,040XX W ARTHINGTON ST,ARSON,41.870014,-87.726102
2,04/15/2019 11:55:00 PM,019XX W MONTEREY AVE,BATTERY,41.691414,-87.668826
3,04/15/2019 11:51:00 PM,076XX S STEWART AVE,BATTERY,41.755613,-87.634421
4,04/15/2019 11:47:00 PM,077XX S NORMAL AVE,NARCOTICS,41.753902,-87.636814


In [19]:
df['Year'] = df['Date'].apply(lambda x: \
                datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').year)
df['Month'] = df['Date'].apply(lambda x: \
                datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').month)
df['Time'] = df['Date'].apply(lambda x: \
                datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').time())

In [20]:
pattern = re.compile('(\d*?)(XX|X)\s+(.*)?')
def get_block_num(x):
    match = pattern.match(x)
    return match.group(1)

df['Block Prefix'] = df['Block'].apply(lambda x: get_block_num(x))
df = df.drop(columns=['Block'])

In [21]:
df.head()

,Date,Primary Type,Latitude,Longitude,Year,Month,Time,Block Prefix
0,04/15/2019 11:59:00 PM,THEFT,42.017499,-87.668188,2019,4,23:59:00,074
1,04/15/2019 11:57:00 PM,ARSON,41.870014,-87.726102,2019,4,23:57:00,040
2,04/15/2019 11:55:00 PM,BATTERY,41.691414,-87.668826,2019,4,23:55:00,019
3,04/15/2019 11:51:00 PM,BATTERY,41.755613,-87.634421,2019,4,23:51:00,076
4,04/15/2019 11:47:00 PM,NARCOTICS,41.753902,-87.636814,2019,4,23:47:00,077


In [22]:
df.dtypes

Date             object
Primary Type     object
Latitude        float64
Longitude       float64
Year              int64
Month             int64
Time             object
Block Prefix     object
dtype: object

In [23]:
df['Primary Type'].nunique(), df['Primary Type'].unique()

(32, array(['THEFT', 'ARSON', 'BATTERY', 'NARCOTICS', 'OTHER OFFENSE',
        'ASSAULT', 'ROBBERY', 'CRIMINAL DAMAGE', 'BURGLARY',
        'CRIMINAL TRESPASS', 'HOMICIDE', 'MOTOR VEHICLE THEFT',
        'DECEPTIVE PRACTICE', 'WEAPONS VIOLATION',
        'CONCEALED CARRY LICENSE VIOLATION', 'PUBLIC PEACE VIOLATION',
        'OFFENSE INVOLVING CHILDREN', 'INTERFERENCE WITH PUBLIC OFFICER',
        'CRIM SEXUAL ASSAULT', 'STALKING', 'PROSTITUTION', 'SEX OFFENSE',
        'KIDNAPPING', 'INTIMIDATION', 'LIQUOR LAW VIOLATION',
        'OTHER NARCOTIC VIOLATION', 'GAMBLING', 'OBSCENITY',
        'HUMAN TRAFFICKING', 'NON-CRIMINAL', 'PUBLIC INDECENCY',
        'NON-CRIMINAL (SUBJECT SPECIFIED)'], dtype=object))

In [24]:
# severity = {
#     'NON-CRIMINAL': 0, 
#     'NON-CRIMINAL (SUBJECT SPECIFIED)': 0,
    
#     'OTHER OFFENSE': 10,

#     'DECEPTIVE PRACTICE': 20, 
#     'LIQUOR LAW VIOLATION': 20,
#     'GAMBLING': 20, 
    
#     'CRIMINAL TRESPASS': 30, 

#     'CONCEALED CARRY LICENSE VIOLATION': 40,
    
#     'WEAPONS VIOLATION': 50,

#     'PUBLIC INDECENCY': 60,
#     'OBSCENITY': 60,

#     'INTIMIDATION': 80, 
#     'PUBLIC PEACE VIOLATION': 80,

#     'MOTOR VEHICLE THEFT',

#     'THEFT', 
#     'ROBBERY', 
#     'BURGLARY',
    
#     'CRIMINAL DAMAGE', 

#     'STALKING', 

#     'PROSTITUTION', 

#     'INTERFERENCE WITH PUBLIC OFFICER',

#     'ARSON', 
#     'OTHER NARCOTIC VIOLATION', 
    
#     'SEX OFFENSE',
    
#     'NARCOTICS', 
#     'BATTERY', 
#     'ASSAULT', 
    
#     'OFFENSE INVOLVING CHILDREN', 

#     'HUMAN TRAFFICKING', 
#     'KIDNAPPING', 
    
#     'CRIM SEXUAL ASSAULT', 
#     'HOMICIDE', 
#     ]}